<a href="https://colab.research.google.com/github/Anurag38/NLP-Practicals/blob/main/Practical%2011%20-%20Sequence%20Tagging%20with%20HMM/Sequence_Tagging_with_HMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time
 
#download the treebank corpus from nltk
nltk.download('treebank')
 
#download the universal tagset from nltk
nltk.download('universal_tagset')
 
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
 
#print the first two sentences along with tags
print(nltk_data[:2])

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')]]


In [2]:
for sent in nltk_data[:2]:
  for tuple in sent:
    print(tuple)

('Pierre', 'NOUN')
('Vinken', 'NOUN')
(',', '.')
('61', 'NUM')
('years', 'NOUN')
('old', 'ADJ')
(',', '.')
('will', 'VERB')
('join', 'VERB')
('the', 'DET')
('board', 'NOUN')
('as', 'ADP')
('a', 'DET')
('nonexecutive', 'ADJ')
('director', 'NOUN')
('Nov.', 'NOUN')
('29', 'NUM')
('.', '.')
('Mr.', 'NOUN')
('Vinken', 'NOUN')
('is', 'VERB')
('chairman', 'NOUN')
('of', 'ADP')
('Elsevier', 'NOUN')
('N.V.', 'NOUN')
(',', '.')
('the', 'DET')
('Dutch', 'NOUN')
('publishing', 'VERB')
('group', 'NOUN')
('.', '.')


In [3]:
train_set,test_set =train_test_split(nltk_data,train_size=0.80,test_size=0.20,random_state = 101)

In [4]:
train_tagged_words = [ tup for sent in train_set for tup in sent ]
test_tagged_words = [ tup for sent in test_set for tup in sent ]
print(len(train_tagged_words))
print(len(test_tagged_words))

80310
20366


In [5]:
#use set datatype to check how many unique tags are present in training data
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)
 
# check total words in vocabulary
vocab = {word for word,tag in train_tagged_words}

12
{'X', 'DET', 'CONJ', 'NOUN', 'PRT', 'NUM', 'ADJ', 'ADV', 'PRON', 'ADP', 'VERB', '.'}


In [6]:
# compute Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
#now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)
 
     
    return (count_w_given_tag, count_tag)

In [7]:
# compute  Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [8]:
# creating t x t transition matrix of tags, t= no of tags
# Matrix(i, j) represents P(jth tag after the ith tag)
 
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
 
print(tags_matrix)

[[7.57255405e-02 5.68902567e-02 1.03786280e-02 6.16951771e-02
  1.85085520e-01 3.07514891e-03 1.76821072e-02 2.57543717e-02
  5.41995019e-02 1.42225638e-01 2.06419379e-01 1.60868734e-01]
 [4.51343954e-02 6.03708485e-03 4.31220367e-04 6.35906279e-01
  2.87480245e-04 2.28546783e-02 2.06410810e-01 1.20741697e-02
  3.30602261e-03 9.91806854e-03 4.02472317e-02 1.73925534e-02]
 [9.33040585e-03 1.23490669e-01 5.48847427e-04 3.49066973e-01
  4.39077942e-03 4.06147093e-02 1.13611415e-01 5.70801310e-02
  6.03732169e-02 5.59824370e-02 1.50384188e-01 3.51262353e-02]
 [2.88252197e-02 1.31063312e-02 4.24540639e-02 2.62344331e-01
  4.39345129e-02 9.14395228e-03 1.25838192e-02 1.68945398e-02
  4.65906132e-03 1.76826611e-01 1.49133503e-01 2.40094051e-01]
 [1.21330721e-02 1.01369865e-01 2.34833662e-03 2.50489235e-01
  1.17416831e-03 5.67514673e-02 8.29745606e-02 9.39334650e-03
  1.76125243e-02 1.95694715e-02 4.01174158e-01 4.50097844e-02]
 [2.02427700e-01 3.57015361e-03 1.42806144e-02 3.51660132e-01
  2

In [9]:
# convert the matrix to a df for better readability
#the table is same as the transition table shown in section 3 of article
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

,X,DET,CONJ,NOUN,PRT,NUM,ADJ,ADV,PRON,ADP,VERB,.
X,0.075726,0.056890,0.010379,0.061695,0.185086,0.003075,0.017682,0.025754,0.054200,0.142226,0.206419,0.160869
DET,0.045134,0.006037,0.000431,0.635906,0.000287,0.022855,0.206411,0.012074,0.003306,0.009918,0.040247,0.017393
CONJ,0.009330,0.123491,0.000549,0.349067,0.004391,0.040615,0.113611,0.057080,0.060373,0.055982,0.150384,0.035126
NOUN,0.028825,0.013106,0.042454,0.262344,0.043935,0.009144,0.012584,0.016895,0.004659,0.176827,0.149134,0.240094
PRT,0.012133,0.101370,0.002348,0.250489,0.001174,0.056751,0.082975,0.009393,0.017613,0.019569,0.401174,0.045010
NUM,0.202428,0.003570,0.014281,0.351660,0.026062,0.184220,0.035345,0.003570,0.001428,0.037487,0.020707,0.119243
ADJ,0.020971,0.005243,0.016893,0.696893,0.011456,0.021748,0.063301,0.005243,0.000194,0.080583,0.011456,0.066019
ADV,0.022886,0.071373,0.006982,0.032196,0.014740,0.029868,0.130721,0.081458,0.012025,0.119472,0.339022,0.139255
PRON,0.088383,0.009567,0.005011,0.212756,0.014123,0.006834,0.070615,0.036902,0.006834,0.022323,0.484738,0.041913
ADP,0.034548,0.320931,0.001012,0.323589,0.001266,0.063275,0.107062,0.014553,0.069603,0.016958,0.008479,0.038724


In [10]:
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [11]:
# Let's test our Viterbi algorithm on a few sample sentences of test dataset
random.seed(1234)      #define a random seed to get same sentences when run multiple times
 
# choose random 10 numbers
rndom = [random.randint(1,len(test_set)) for x in range(10)]
 
# list of 10 sents on which we test the model
test_run = [test_set[i] for i in rndom]
 
# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]
 
# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

In [12]:
#Here We will only test 10 sentences to check the accuracy
#as testing the whole training set takes huge amount of time
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  39.675126791000366
Viterbi Algorithm Accuracy:  94.25837320574163


In [13]:
#To improve the performance,we specify a rule base tagger for unknown words 
# specify patterns for tagging
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense 
    (r'.*es$', 'VERB'),               # verb    
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'\*T?\*?-[0-9]+$', 'X'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'.*', 'NOUN')                   # nouns
]
 
# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [14]:
#modified Viterbi to include rule based tagger in it
def Viterbi_rule_based(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1]       
        
         
        if(pmax==0):
            state_max = rule_based_tagger.tag([word])[0][1] # assign based on rule based tagger
        else:
            if state_max != 'X':
                # getting state for which probability is maximum
                state_max = T[p.index(pmax)]                
             
         
        state.append(state_max)
    return list(zip(words, state))

In [15]:
#test accuracy on subset of test data 
start = time.time()
tagged_seq = Viterbi_rule_based(test_tagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  39.2841374874115
Viterbi Algorithm Accuracy:  97.1291866028708


In [17]:
#Check how a sentence is tagged by the two POS taggers
#and compare them
test_sent="Booking a Train Ticket"
pred_tags_rule=Viterbi_rule_based(test_sent.split())
pred_tags_withoutRules= Viterbi(test_sent.split())
print(pred_tags_rule)
print(pred_tags_withoutRules)


[('Booking', 'VERB'), ('a', 'DET'), ('Train', 'NOUN'), ('Ticket', 'NOUN')]
[('Booking', 'X'), ('a', 'DET'), ('Train', 'X'), ('Ticket', 'X')]
